In [1]:
import numpy as np
import pandas as pd
from processing import YouScanProcessing as ysp
from gspread_pandas import Spread, Client
import gspread_pandas
import os.path
import os

In [2]:
df = pd.read_excel('gazp.xlsx')

## Оценка количества и динамики публикаций с упоминанием Заказчика и упоминанием Заказчика, а также программы газификации в главной роли

In [3]:
publications_count = df.groupby('Дата').agg({'Дата': 'count'}) \
                        .rename(columns={'Дата': 'Кол-во'}) \
                        .sort_index(ascending=True)
publications_count = publications_count \
                .append(pd.DataFrame({'Кол-во': publications_count.sum()[0]}, 
                                     index=['Всего']))
publications_count

,Кол-во
14.05.2021,247
15.05.2021,139
16.05.2021,121
Всего,507


## География распределения публикаций о Заказчике

In [4]:
city = df.groupby('Город').agg({'Город': 'count'}) \
            .rename(columns={'Город': 'Кол-во'}) \
            .sort_values('Кол-во',ascending=False).iloc[:10]
city = city.append(pd.DataFrame({'Кол-во': df.groupby('Город')['Дата']
                                 .count().sum()}, index=['Всего']))
city

,Кол-во
Москва,17
Санкт-Петербург,13
Воскресенск,10
Липецк,9
Рязань,8
Кострома,7
Севастополь,7
Александров,6
Сочи,6
Челябинск,5


## Анализ тональности публикаций о Заказчике (города)

In [5]:
calc_cols = ['Negative', 'Neutral', 'Positive', 'Total']
per_cols = ['Negative %', 'Neutral %', 'Positive %']
tonality_columns = {'Negative': 'Негативная', 
                    'Neutral': 'Нейтральная', 
                    'Positive': 'Позитивная', 
                    'Total': 'Всего', 
                    'Negative %': 'Негативная %', 
                    'Neutral %': 'Нейтральная %', 
                    'Positive %': 'Позитивная %'}

In [6]:
city_tonality = ysp.tonality(df, city.index.to_list()[:-1], 'Город')
city_tonality = ysp.tonality_percent(city_tonality, total_types=False) \
                   .rename(columns=tonality_columns)
city_tonality

,Негативная,Нейтральная,Позитивная,Всего,Негативная %,Нейтральная %,Позитивная %
Москва,0,17,0,17,0.00,100.00,0.0
Санкт-Петербург,1,12,0,13,7.69,92.31,0.0
Воскресенск,0,10,0,10,0.00,100.00,0.0
Липецк,0,9,0,9,0.00,100.00,0.0
Рязань,0,8,0,8,0.00,100.00,0.0
Кострома,7,0,0,7,100.00,0.00,0.0
Севастополь,2,5,0,7,28.57,71.43,0.0
Александров,4,2,0,6,66.67,33.33,0.0
Сочи,0,6,0,6,0.00,100.00,0.0
Челябинск,0,5,0,5,0.00,100.00,0.0


## Анализ тональности публикаций о Заказчике (общая)

In [7]:
total_tonality = ysp.tonality(df, df['Дата'].unique(), 'Дата')
total_tonality = ysp.tonality_percent(total_tonality) \
                    .rename(columns=tonality_columns)
total_tonality

,Негативная,Нейтральная,Позитивная,Всего,Негативная %,Нейтральная %,Позитивная %
14.05.2021,12,235,0,247,4.86,95.14,0.0
15.05.2021,9,130,0,139,6.47,93.53,0.0
16.05.2021,18,103,0,121,14.88,85.12,0.0
Total types,39,468,0,507,7.69,92.31,0.0


## Рейтинги наиболее активных сообществ в освещении деятельности Заказчика

In [8]:
authors = df.groupby(['Источник','Автор']) \
            .agg({'Автор': 'count'}).rename(columns={'Автор': 'Кол-во'}) \
            .sort_values('Кол-во',ascending=False).iloc[:25]
authors

Кол-во
Источник        Автор                                  
vk.com          Denis Sarantsev                       5
advis.ru        advis.ru                              4
zen.yandex.ru   CHEM.RU                               4
telegram.me     Вести Сочи                            4
facebook.com    Konstantin B. Getmansky  Cat          4
goldring.ru     Золотое кольцо                        4
telegram.me     Дайджест: ТЭК и экология              3
                Зелёная Тележенька                    3
                Мониторинг 38 Иркутская область       3
                ToyRus                                3
                Глобальная энергия                    2
vk.com          Ольга Курочкина                       2
telegram.me     Такси закажите в личку                2
youtube.com     Дмитрий Жихарев                       2
vk.com          Юридическая клиника ТулГУ             2
                Константин Комков                     2
instagram.com   СВЕЖИЕ НОВОСТИ СЕВАСТОПОЛЯ            2
youtube.com     Julia Galkina                         2
vk.com          Инна Альбертова                       2
youtube.com     Вести Карачаево-Черкесия              2
                Вести Севастополь                     2
                Воскресенск Онлайн TV                 2
vk.com          Максим Перепелица                     2
youtube.com     Свободные                             2
energyland.info EnergyLand.info                       2

## Топ источников

In [9]:
sources = df.groupby('Источник') \
            .agg({'Источник': 'count'}).rename(columns={'Источник': 'Кол-во'}) \
            .sort_values('Кол-во', ascending=False).iloc[:10]
sources = sources.append(pd.DataFrame({'Кол-во': df.groupby('Источник')
                                           .agg({'Источник': 'count'})
                                           .sum()[0]}, index=['Всего']))
sources

,Кол-во
vk.com,258
telegram.me,88
youtube.com,40
ok.ru,25
facebook.com,25
instagram.com,19
zen.yandex.ru,9
advis.ru,4
goldring.ru,4
energyland.info,4


## Анализ тональности публикаций о Заказчике (источники)

In [10]:
sources_tonality = ysp.tonality(df, sources.index.to_list()[:-1], 'Источник')
sources_tonality = ysp.tonality_percent(sources_tonality, total_types=False) \
                      .rename(columns=tonality_columns)
sources_tonality

,Негативная,Нейтральная,Позитивная,Всего,Негативная %,Нейтральная %,Позитивная %
vk.com,27,231,0,258,10.47,89.53,0.0
telegram.me,2,86,0,88,2.27,97.73,0.0
youtube.com,4,36,0,40,10.00,90.00,0.0
ok.ru,2,23,0,25,8.00,92.00,0.0
facebook.com,3,22,0,25,12.00,88.00,0.0
instagram.com,0,19,0,19,0.00,100.00,0.0
zen.yandex.ru,0,9,0,9,0.00,100.00,0.0
advis.ru,0,4,0,4,0.00,100.00,0.0
goldring.ru,0,4,0,4,0.00,100.00,0.0
energyland.info,0,4,0,4,0.00,100.00,0.0


## Анализ тональности публикаций о Заказчике (теги)

In [11]:
tags_2 = [col for col in df.columns if col.startswith('2_')]

def del_tag_num(name):
    return name.replace('2_','')

In [12]:
tags_tonality = pd.DataFrame()
for tag in tags_2:
    temp_df = ysp.tonality(df, [tag], tag)
    tags_tonality = tags_tonality.append(temp_df) \
                        .sort_values(by=['Total'], ascending=False)
tags_tonality.iloc[0] = tags_tonality.iloc[0] - tags_tonality[1:].sum()
tags_tonality = ysp.tonality_percent(tags_tonality, total_types=False) \
                   .rename(columns=tonality_columns) \
                   .rename(index=del_tag_num)
tags_tonality

,Негативная,Нейтральная,Позитивная,Всего,Негативная %,Нейтральная %,Позитивная %
Правительственная программа,21,373,0,394,5.33,94.67,0.0
Руководство/ Совещания/ Партнёрство,0,87,0,87,0.00,100.00,0.0
Стоимость/Оплата,7,4,0,11,63.64,36.36,0.0
Жалоба,11,0,0,11,100.00,0.00,0.0
Вопросы о подключении,0,4,0,4,0.00,100.00,0.0


## Рейтинг цитируемости спикеров, упоминаемых в контексте новостей о программе и ее проектах

In [13]:
speak ={'путин':'Владимир Путин', 'новак':'Александр Новак', 
        'миллер':'Алексей Миллер', 'густов':'Сергей Густов', 
        'шульгинов':'Николай Шульгинов', 'сорокин':'Павел Сорокин', 
        'завальный':'Павел Завальный'}
temp_dict = {}
for key in speak.keys():
    temp_dict[key] = (df['Заголовок'].fillna('').apply(lambda x: x.lower()).str.contains(key).sum() + 
                    df['Дайджест текста'].apply(lambda x: x.lower()).str.contains(key).sum())

speakes = pd.DataFrame(temp_dict,index=['Кол-во']) \
                            .rename(columns=speak) \
                            .transpose().sort_values('Кол-во', ascending=False)
speakes = speakes.append(pd.DataFrame({'Кол-во': speakes.sum()[0]}, index=['Всего']))
speakes

,Кол-во
Владимир Путин,89
Александр Новак,63
Сергей Густов,45
Алексей Миллер,18
Николай Шульгинов,8
Павел Сорокин,0
Павел Завальный,0
Всего,223


## Статистика по охвату аудитории

In [14]:
d = {
    'Охват': df.groupby('Профиль места публикации') \
            .agg({'Подписчики места публикации': 'mean'}).sum()[0],
    'Упоминания': df.count()[0],
    'Негативные': df.query("Тональность == 'Негативная'").agg({'Тональность': 'count'})[0],
    'Нейтральные': df.query("Тональность == 'Нейтральная'").agg({'Тональность': 'count'})[0],
    'Позитивные': df.query("Тональность == 'Позитивная'").agg({'Тональность': 'count'})[0],
    'Вовлечение': df[['Репосты','Комментарии','Сумма всех реакций']].sum().sum()
}
stats = pd.DataFrame(d, index=['Кол-во']).transpose().astype(int)
stats

,Кол-во
Охват,11038714
Упоминания,507
Негативные,39
Нейтральные,468
Позитивные,0
Вовлечение,3357


## Распределение публикаций по уровню и видам сообществ (федеральные и региональные)

In [15]:
source_type = df.groupby('Тип источника') \
                .agg({'Дата': 'count'}).sort_values('Дата',ascending=False) \
                .rename(columns={'Дата': 'Кол-во'})
source_type = source_type.append(pd.DataFrame({'Кол-во': source_type.sum()[0]}, index=['Всего']))
source_type

,Кол-во
Соц. сеть,371
Мессенджеры,85
СМИ,31
Блог,17
Форум,2
Отзывы,1
Всего,507


# TELEGRAM

In [18]:
c = gspread_pandas.conf.get_config(conf_dir='./', file_name='google_secret.json')
spread = Spread('ALL Ежедневный / Еженедельный Рабочий файл Газпром', config=c)
spread.sheets

[<Worksheet 'Дайджест' id:1416872051>,
 <Worksheet 'gazp' id:1201477038>,
 <Worksheet 'Графики' id:703305161>,
 <Worksheet 'telegram' id:1578170012>,
 <Worksheet 'topicНехватка_денег.xlsx' id:337983581>]

In [19]:
for file in os.walk(os.getcwd()):    
    topics = [f for f in file[2] if f.startswith('topic')]
    break
print(topics)

['topicНехватка_денег.xlsx']


In [26]:
link_list = []
for topic in topics:
    link_df = pd.read_excel(topic)
    link_df['URLtext'] = link_df.URL.apply(lambda x: '(' + x + ');')
    link_list += link_df.URL.to_list()
    spread.df_to_sheet(link_df[['URL', 'URLtext']], sheet=topic, start='A1', index=False, replace=True)

In [8]:
telegram = df.query('Источник == "telegram.me"')[['Дайджест текста','Автор' ,'Тональность','Подписчики места публикации','URL']]
telegram['Дайджест текста'] = telegram['Дайджест текста'].str.replace('\n',' ')
telegram['URL'] = telegram.URL.apply(lambda x: x if x not in link_list else np.nan)
telegram = telegram.dropna().sort_values('Подписчики места публикации',ascending=False)
teleg = {}
for row in telegram.query('Тональность == "Позитивная"').index:
        teleg[row] = f"{telegram['Автор'].loc[row]} (Подписчики: {telegram['Подписчики места публикации'].loc[row].astype(int)}), Цитата: \"{telegram['Дайджест текста'].loc[row]}\". ({telegram['URL'].loc[row]});"
t_pos = pd.DataFrame(teleg, index=['Позитивная']).transpose()
        
teleg = {}
for row in telegram.query('Тональность == "Нейтральная"').index:
        teleg[row] = f"{telegram['Автор'].loc[row]} (Подписчики: {telegram['Подписчики места публикации'].loc[row].astype(int)}), Цитата: \"{telegram['Дайджест текста'].loc[row]}\". ({telegram['URL'].loc[row]});"
t_neu = pd.DataFrame(teleg, index=['Нейтральная']).transpose() 
        
teleg = {}
for row in telegram.query('Тональность == "Негативная"').index:
        teleg[row] = f"{telegram['Автор'].loc[row]} (Подписчики: {telegram['Подписчики места публикации'].loc[row].astype(int)}), Цитата: \"{telegram['Дайджест текста'].loc[row]}\". ({telegram['URL'].loc[row]});"
t_neg = pd.DataFrame(teleg, index=['Негативная']).transpose() 

# ВЫГРУЗКА

In [134]:
spread.df_to_sheet(publications_count, sheet='gazp', start='A2') 
spread.df_to_sheet(city, sheet='gazp', start='A11') 
spread.df_to_sheet(authors, sheet='gazp', start='A26')

spread.df_to_sheet(sources, sheet='gazp', start='D2')
spread.df_to_sheet(speakes, sheet='gazp', start='D17') 
spread.df_to_sheet(stats, sheet='gazp', start='D28')
spread.df_to_sheet(source_type, sheet='gazp', start='D38')

spread.df_to_sheet(city_tonality, sheet='gazp', start='G2') 
spread.df_to_sheet(total_tonality, sheet='gazp', start='G17') 
spread.df_to_sheet(sources_tonality, sheet='gazp', start='G31') 

spread.df_to_sheet(tags_tonality, sheet='gazp', start='P2') 

In [186]:
spread.df_to_sheet(t_pos, sheet='telegram', start='A1', index=False, replace=True) 
spread.df_to_sheet(t_neu, sheet='telegram', start='B1', index=False) 
spread.df_to_sheet(t_neg, sheet='telegram', start='C1', index=False) 